In [1]:
DATA_NAME = 'spaceNet-full' 
TRANSFORM = 'fourier'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_fourier_full_spaceNet.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,1168.36,-0.01,0.01,0.02,0.00,0.00,0.01,0.03,-0.00,0.01,...,0.00,0.00,-0.0,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00
3,-0.01,833.73,0.00,-0.02,0.02,-0.02,0.00,0.01,-0.00,-0.00,...,-0.00,0.00,0.0,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00
4,0.01,0.00,625.00,0.02,0.01,-0.00,0.01,-0.01,0.01,0.00,...,-0.00,0.00,0.0,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00
5,0.02,-0.02,0.02,467.15,-0.02,0.02,-0.00,-0.01,0.00,0.00,...,-0.00,-0.00,0.0,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00
6,0.00,0.02,0.01,-0.02,319.09,0.00,-0.00,-0.00,-0.01,0.00,...,0.00,-0.00,0.0,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00
7,0.00,-0.02,-0.00,0.02,0.00,195.20,-0.00,0.00,0.00,-0.00,...,-0.00,0.00,-0.0,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00
8,0.01,0.00,0.01,-0.00,-0.00,-0.00,126.60,0.00,-0.00,0.00,...,0.00,0.00,-0.0,0.00,-0.00,0.00,-0.00,-0.00,-0.00,-0.00
9,0.03,0.01,-0.01,-0.01,-0.00,0.00,0.00,76.08,-0.00,-0.00,...,0.00,0.00,-0.0,0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00
10,-0.00,-0.00,0.01,0.00,-0.01,0.00,-0.00,-0.00,47.60,0.00,...,0.00,-0.00,0.0,-0.00,0.00,0.00,-0.00,0.00,0.00,0.00
11,0.01,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,29.45,...,0.00,0.00,0.0,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,15,16,17,18,19,20,21,22,23,24
2,1.00000,-0.00001,0.00002,0.00003,0.00000,0.00001,0.00004,0.00010,-0.00000,0.00003,...,0.00001,0.00001,-0.00002,0.00007,0.00005,-0.00000,0.00002,-0.00001,0.00004,-0.00002
3,-0.00001,1.00000,0.00000,-0.00003,0.00003,-0.00006,0.00001,0.00003,-0.00002,-0.00002,...,-0.00001,0.00003,0.00000,0.00003,0.00003,-0.00001,0.00003,-0.00004,-0.00004,-0.00000
4,0.00002,0.00000,1.00000,0.00004,0.00002,-0.00000,0.00003,-0.00005,0.00007,0.00000,...,-0.00004,0.00000,0.00004,-0.00004,0.00000,0.00001,0.00000,-0.00002,0.00002,0.00000
5,0.00003,-0.00003,0.00004,1.00000,-0.00004,0.00005,-0.00000,-0.00003,0.00001,0.00001,...,-0.00002,-0.00002,0.00001,-0.00001,0.00004,0.00000,0.00001,-0.00002,0.00005,0.00003
6,0.00000,0.00003,0.00002,-0.00004,1.00000,0.00000,-0.00000,-0.00001,-0.00004,0.00004,...,0.00002,-0.00000,0.00002,-0.00000,0.00002,0.00004,-0.00002,0.00003,-0.00004,-0.00001
7,0.00001,-0.00006,-0.00000,0.00005,0.00000,1.00000,-0.00001,0.00000,0.00003,-0.00002,...,-0.00004,0.00002,-0.00003,0.00008,-0.00003,0.00004,0.00001,-0.00005,0.00000,-0.00003
8,0.00004,0.00001,0.00003,-0.00000,-0.00000,-0.00001,1.00000,0.00001,-0.00005,0.00006,...,0.00001,0.00004,-0.00001,0.00000,-0.00000,0.00000,-0.00002,-0.00007,-0.00003,-0.00000
9,0.00010,0.00003,-0.00005,-0.00003,-0.00001,0.00000,0.00001,1.00000,-0.00004,-0.00002,...,0.00001,0.00000,-0.00001,0.00005,-0.00002,0.00003,0.00002,0.00002,-0.00003,-0.00006
10,-0.00000,-0.00002,0.00007,0.00001,-0.00004,0.00003,-0.00005,-0.00004,1.00000,0.00005,...,0.00001,-0.00000,0.00001,-0.00002,0.00001,0.00002,-0.00002,0.00002,0.00001,0.00002
11,0.00003,-0.00002,0.00000,0.00001,0.00004,-0.00002,0.00006,-0.00002,0.00005,1.00000,...,0.00001,0.00001,0.00005,-0.00001,0.00004,0.00005,-0.00004,-0.00001,-0.00006,0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.09797587688767795

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.95550507e+04 3.02360983e+04 1.69276290e+04 9.41336037e+03
 4.37919914e+03 1.63661019e+03 6.85449896e+02 2.46927148e+02
 9.61989366e+01 3.66916171e+01 1.52791267e+01 6.86990449e+00
 2.97777586e+00 1.21103111e+00 4.99715741e-01 1.75350540e-01
 6.01745143e-02 1.59520826e-02 4.80897624e-03 1.17497626e-03
 4.94148783e-04 2.33424762e-04 1.60041599e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.997157,-0.063474,-0.031795,-0.019996,-0.012362,-0.007178,-0.004566,-0.002704,-0.001704,-0.001048,...,-0.000193,-0.000125,-0.000076,-0.000042,-0.000022,-0.000013,-0.000006,-0.000004,-0.000003,-0.000002
1,0.059552,0.993916,-0.081152,-0.037314,-0.020159,-0.011135,-0.006948,-0.004101,-0.002567,-0.001584,...,-0.000292,-0.000186,-0.000113,-0.000065,-0.000034,-0.000020,-0.000009,-0.000007,-0.000005,-0.000003
2,0.034012,0.074680,0.991972,-0.087450,-0.034056,-0.016890,-0.010251,-0.006015,-0.003698,-0.002293,...,-0.000426,-0.000272,-0.000161,-0.000098,-0.000050,-0.000028,-0.000014,-0.000009,-0.000006,-0.000005
3,0.023846,0.040469,0.080451,0.992493,-0.071821,-0.027272,-0.015678,-0.008955,-0.005491,-0.003378,...,-0.000624,-0.000404,-0.000239,-0.000143,-0.000072,-0.000041,-0.000020,-0.000014,-0.000009,-0.000007
4,0.015710,0.023722,0.036205,0.065491,0.994621,-0.057172,-0.026893,-0.014343,-0.008649,-0.005249,...,-0.000960,-0.000623,-0.000370,-0.000221,-0.000113,-0.000063,-0.000032,-0.000020,-0.000015,-0.000011
5,0.009393,0.013553,0.018835,0.027339,0.051812,0.995121,-0.067727,-0.026905,-0.015055,-0.008966,...,-0.001622,-0.001042,-0.000626,-0.000363,-0.000193,-0.000106,-0.000053,-0.000036,-0.000025,-0.000018
6,0.006427,0.009119,0.012363,0.017109,0.027374,0.063095,0.994899,-0.060907,-0.027683,-0.015382,...,-0.002686,-0.001731,-0.001037,-0.000613,-0.000319,-0.000178,-0.000089,-0.000060,-0.000042,-0.000030
7,0.003976,0.005622,0.007576,0.010239,0.015429,0.027831,0.055645,0.994655,-0.070093,-0.030073,...,-0.004683,-0.003022,-0.001803,-0.001059,-0.000556,-0.000308,-0.000152,-0.000100,-0.000073,-0.000054
8,0.002664,0.003745,0.004982,0.006716,0.009970,0.016844,0.028344,0.064475,0.993668,-0.073180,...,-0.008094,-0.005198,-0.003087,-0.001829,-0.000948,-0.000529,-0.000263,-0.000172,-0.000122,-0.000089
9,0.001729,0.002435,0.003245,0.004347,0.006384,0.010602,0.016810,0.030949,0.065991,0.992050,...,-0.014142,-0.008955,-0.005276,-0.003125,-0.001613,-0.000901,-0.000451,-0.000296,-0.000213,-0.000153


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0028425864848922178,
 0.0060839196359767644,
 0.008027506918670269,
 0.007506533218458733,
 0.005379358745731477,
 0.004879155464331153,
 0.005101169208481582,
 0.0053447095850827075,
 0.006332214130364888,
 0.007950023622437019,
 0.010899258498115572,
 0.013410571670701255,
 0.014668749226212197,
 0.016720228703789197,
 0.01792449702312726,
 0.01869754158737713,
 0.019394559945030476,
 0.023637029816896304,
 0.031720887152039245,
 0.0690516686800775,
 0.1533682070746083,
 0.29687484217269966,
 0.28471068807896494]